In [12]:
import streamlit as st
from streamlit_webrtc import webrtc_streamer
import av
import cv2 
import numpy as np 
import mediapipe as mp 
from keras.models import load_model
import webbrowser

In [13]:
model  = load_model("./Models/model.h5")
label = np.load("./Models/labels.npy")
holistic = mp.solutions.holistic
hands = mp.solutions.hands
holis = holistic.Holistic()
drawing = mp.solutions.drawing_utils

In [14]:
st.header("Emotion Based Music Recommender")

2024-10-19 02:52:41.263 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-19 02:52:41.263 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [15]:
if "run" not in st.session_state:
	st.session_state["run"] = "true"

try:
	emotion = np.load("emotion.npy")[0]
except:
	emotion=""

2024-10-19 02:52:41.310 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [16]:
if not(emotion):
	st.session_state["run"] = "true"
else:
	st.session_state["run"] = "false"

2024-10-19 02:52:41.395 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-19 02:52:41.398 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [17]:
class EmotionProcessor:
	def recv(self, frame):
		frm = frame.to_ndarray(format="bgr24")

		##############################
		frm = cv2.flip(frm, 1)

		res = holis.process(cv2.cvtColor(frm, cv2.COLOR_BGR2RGB))

		lst = []

		if res.face_landmarks:
			for i in res.face_landmarks.landmark:
				lst.append(i.x - res.face_landmarks.landmark[1].x)
				lst.append(i.y - res.face_landmarks.landmark[1].y)

			if res.left_hand_landmarks:
				for i in res.left_hand_landmarks.landmark:
					lst.append(i.x - res.left_hand_landmarks.landmark[8].x)
					lst.append(i.y - res.left_hand_landmarks.landmark[8].y)
			else:
				for i in range(42):
					lst.append(0.0)

			if res.right_hand_landmarks:
				for i in res.right_hand_landmarks.landmark:
					lst.append(i.x - res.right_hand_landmarks.landmark[8].x)
					lst.append(i.y - res.right_hand_landmarks.landmark[8].y)
			else:
				for i in range(42):
					lst.append(0.0)

			lst = np.array(lst).reshape(1,-1)

			pred = label[np.argmax(model.predict(lst))]

			print(pred)
			cv2.putText(frm, pred, (50,50),cv2.FONT_ITALIC, 1, (255,0,0),2)

			np.save("emotion.npy", np.array([pred]))

			
		drawing.draw_landmarks(frm, res.face_landmarks, holistic.FACEMESH_TESSELATION,
								landmark_drawing_spec=drawing.DrawingSpec(color=(0,0,255), thickness=-1, circle_radius=1),
								connection_drawing_spec=drawing.DrawingSpec(thickness=1))
		drawing.draw_landmarks(frm, res.left_hand_landmarks, hands.HAND_CONNECTIONS)
		drawing.draw_landmarks(frm, res.right_hand_landmarks, hands.HAND_CONNECTIONS)


		##############################

		return av.VideoFrame.from_ndarray(frm, format="bgr24")

In [18]:
lang = st.text_input("Language")
singer = st.text_input("singer")

2024-10-19 02:52:41.465 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-19 02:52:41.467 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-19 02:52:41.469 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-19 02:52:41.470 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-19 02:52:41.472 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-19 02:52:41.473 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-19 02:52:41.475 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-19 02:52:41.477 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [19]:
if lang and singer and st.session_state["run"] != "false":
	webrtc_streamer(key="key", desired_playing_state=True,
				video_processor_factory=EmotionProcessor)

In [20]:
btn = st.button("Recommend me songs")

if btn:
	if not(emotion):
		st.warning("Please let me capture your emotion first")
		st.session_state["run"] = "true"
	else:
		webbrowser.open(f"https://www.youtube.com/results?search_query={lang}+{emotion}+song+{singer}")
		np.save("emotion.npy", np.array([""]))
		st.session_state["run"] = "false"

2024-10-19 02:52:41.602 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-19 02:52:41.609 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-19 02:52:41.611 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-19 02:52:41.614 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-19 02:52:41.615 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
